In [1]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
import torch
from transformers import OPTForCausalLM, AutoModelForCausalLM, AutoTokenizer

from datasets import load_dataset
from datasets.iterable_dataset import IterableDataset
from transformers import OPTForCausalLM
from chemlactica.utils.model_utils import load_model
from chemlactica.utils.utils import get_tokenizer
import scipy
import numpy as np
from sklearn.metrics import root_mean_squared_error
from rdkit import Chem

/auto/home/menuab/miniforge3/envs/gemma_env_new/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gal_tokenizer = AutoTokenizer.from_pretrained("/auto/home/menuab/code/ChemLactica/chemlactica/tokenizer/ChemLacticaTokenizer66")
gem_tokenizer = AutoTokenizer.from_pretrained("/auto/home/menuab/code/ChemLactica/chemlactica/tokenizer/GemmaTokenizer")
len(gal_tokenizer), len(gem_tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


(50066, 256000)

In [53]:
path = "/nfs/dgx/raid/chem/checkpoints/galactica-125m/1f289ff103034364bd27e1c3/b9a3b47cdd244547abfac334"
!ls "/nfs/dgx/raid/chem/checkpoints/galactica-125m/1f289ff103034364bd27e1c3/b9a3b47cdd244547abfac334"


last


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [123]:
model_125m_20k_9954 = "/nfs/dgx/raid/chem/checkpoints/facebook/galactica-125m/9954e52e400b43d18d3a40f6/checkpoint-20480"
model_2b_11k_d6e6 = "/nfs/dgx/raid/chem/checkpoints/google/gemma-2b/d6e6a76e91814ad68d5fa264/checkpoint-11000"
model_125m_18k_1f28 = "/nfs/dgx/raid/chem/checkpoints/facebook/galactica-125m/1f289ff103034364bd27e1c3/checkpoint-18000/"
model_2b_12k_0717 = "/nfs/dgx/raid/chem/checkpoints/h100/google/gemma-2b/0717d445bcf44e31b2887892/checkpoint-12000"


In [4]:
hPPB = "/nfs/dgx/raid/chem/checkpoints/galactica-125m/1f289ff103034364bd27e1c3/b1b66dbf0b834f1c9da0a444"
rPPB = "/nfs/dgx/raid/chem/checkpoints/galactica-125m/1f289ff103034364bd27e1c3/b9da010c01844cefb8b03426"
HLM = "/nfs/dgx/raid/chem/checkpoints/galactica-125m/1f289ff103034364bd27e1c3/803746806706444c8f36d6e9"
MD1 = "/nfs/dgx/raid/chem/checkpoints/galactica-125m/1f289ff103034364bd27e1c3/3261ee682d58403bb8b4b29a"
Sol = "/nfs/dgx/raid/chem/checkpoints/galactica-125m/1f289ff103034364bd27e1c3/f9218c28afc64175b07813fd"
RLM = "/nfs/dgx/raid/chem/checkpoints/galactica-125m/1f289ff103034364bd27e1c3/310a879774b842d4af6c3822"
freesolv = "/nfs/dgx/raid/chem/checkpoints/galactica-125m/1f289ff103034364bd27e1c3/07677b1fa3014ab1bf445f16"
esol = "/nfs/dgx/raid/chem/checkpoints/galactica-125m/1f289ff103034364bd27e1c3/421bf81ba8754fe9ac651eb0"

In [32]:
dataset = load_dataset(
                "/auto/home/menuab/code/sft_data/ADME_RLM"
            )

In [ ]:
HLM = "/nfs/dgx/raid/chem/checkpoints/google/gemma-2b/c247f1a414ae4bb9ad855748"
RLM = "/nfs/dgx/raid/chem/checkpoints/google/gemma-2b/6a1f3428e7004e35a58e85c3"
MD1 = "/nfs/dgx/raid/chem/checkpoints/google/gemma-2b/27190d95359b4232ad86a67b"
rPPB = "/nfs/dgx/raid/chem/checkpoints/google/gemma-2b/27190d95359b4232ad86a67b"
hPPB = "/nfs/dgx/raid/chem/checkpoints/google/gemma-2b/8898500c69594b6880765375"
Sol = "/nfs/dgx/raid/chem/checkpoints/google/gemma-2b/1f37207f5fc048f8990def13"

In [54]:
dataset = load_dataset("gayane/lipo")
dataset

DatasetDict({
    train: Dataset({
        features: ['smiles', 'activity'],
        num_rows: 3360
    })
    validation: Dataset({
        features: ['smiles', 'activity'],
        num_rows: 420
    })
    test: Dataset({
        features: ['smiles', 'activity'],
        num_rows: 420
    })
})

In [122]:
path = "/nfs/dgx/raid/chem/checkpoints/google/gemma-2b/8898500c69594b6880765375"
!ls "/nfs/dgx/raid/chem/checkpoints/google/gemma-2b/8898500c69594b6880765375"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


last


In [55]:
model = AutoModelForCausalLM.from_pretrained(
    path + "/last", torch_dtype=torch.bfloat16, attn_implementation="flash_attention_2"
).to("cuda:1").eval()
model.device, model.dtype

(device(type='cuda', index=1), torch.bfloat16)

In [56]:
tokenizer = gal_tokenizer
ground_truths, gens, diffs = [],[],[]
invalids = 0
for sample in dataset['test']:
    ground_truth = round(sample['activity'], 2)
    prompt = f"<bos>[START_SMILES]{sample['smiles']}[END_SMILES][PROPERTY]activity"
    len_prompt = len(prompt)
    prompt = tokenizer(prompt, return_tensors="pt").to(model.device)
    out = model.generate(prompt.input_ids, do_sample=False, eos_token_id=tokenizer.encode('[/PROPERTY]')[0], max_new_tokens=100)
    out = tokenizer.batch_decode(out)[0]
    try:
        gen = float(out[out.find("activity ") + len("activity "):out.find("[/PROPERTY]")])
        diff = abs(ground_truth - gen)
        print("GT:", ground_truth, "Gen:", gen, "diff:", round(diff,2), out )
        ground_truths.append(ground_truth)
        gens.append(gen)
        diffs.append(diff)
    except:
        print('***')
        print("GT:", ground_truth, out)
        invalids += 1
        pass

GT: 0.44 Gen: 1.02 diff: 0.58 </s>[START_SMILES]Cc1ccccc1NC(=O)CCS(=O)(=O)c2ccc(Br)s2[END_SMILES][PROPERTY]activity 1.02[/PROPERTY]
GT: -0.81 Gen: -0.89 diff: 0.08 </s>[START_SMILES]COCCNC(=O)c1cccc(CN2NC(=O)C3=C(C2=O)C(=O)c4ccc(Cl)cc4N3)c1[END_SMILES][PROPERTY]activity -0.89[/PROPERTY]
GT: 0.48 Gen: 0.69 diff: 0.21 </s>[START_SMILES]OC(=O)c1[nH]c2ccccc2c1CCCOc3cccc4ccccc34[END_SMILES][PROPERTY]activity 0.69[/PROPERTY]
GT: 1.43 Gen: 0.85 diff: 0.58 </s>[START_SMILES]COc1ccc2nc(C)cc(OCC(=O)Nc3ccccc3OC)c2c1[END_SMILES][PROPERTY]activity 0.85[/PROPERTY]
GT: -1.57 Gen: -1.69 diff: 0.12 </s>[START_SMILES]CCS(=O)(=O)c1ccc(Oc2c(C)n(CC(=O)O)c3ccc(cc23)C(F)(F)F)cc1[END_SMILES][PROPERTY]activity -1.69[/PROPERTY]
GT: -0.05 Gen: -0.1 diff: 0.05 </s>[START_SMILES]CN1CCN(CC1)c2ccc3N=CN(C(=O)c3c2)c4cc(NC(=O)c5cncs5)ccc4C[END_SMILES][PROPERTY]activity -0.1[/PROPERTY]
GT: 0.35 Gen: 0.44 diff: 0.09 </s>[START_SMILES]CCCC[C@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CC(C)C)NC(=O)C)C=O[END_SMILES][PROPERTY]activit

In [51]:
r, p = scipy.stats.pearsonr(np.array(ground_truths), np.array(gens))
r, p

(0.9531190418772819, 1.8815465993576179e-59)

In [57]:
root_mean_squared_error(ground_truths, gens)

0.5123914519193309

In [128]:
Chem.MolToSmiles(Chem.MolFromSmiles("[START_SMILES]Cc1ccc(cc1)C=O[END_SMILES][PROPERTY]activity -0.15[/PROPERTY]"))

[22:07:33] SMILES Parse Error: syntax error while parsing: [START_SMILES]Cc1ccc(cc1)C=O[END_SMILES][PROPERTY]activity
[22:07:33] SMILES Parse Error: Failed parsing SMILES '[START_SMILES]Cc1ccc(cc1)C=O[END_SMILES][PROPERTY]activity' for input: '[START_SMILES]Cc1ccc(cc1)C=O[END_SMILES][PROPERTY]activity'


ArgumentError: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(RDKit::ROMol mol, RDKit::SmilesWriteParams params)

In [ ]:
plt.scatter(ground_truths, gens, alpha=0.1)
plt.plot((0,3),(0,3))

NameError: name 'plt' is not defined

In [ ]:
gens

[0.68,
 0.68,
 0.68,
 0.68,
 0.68,
 1.21,
 0.68,
 0.68,
 1.01,
 0.68,
 0.68,
 0.68,
 1.22,
 0.68,
 1.22,
 0.68,
 0.68,
 0.68,
 1.01,
 0.68,
 0.68,
 0.68,
 1.11,
 0.68,
 0.68,
 0.68,
 0.68,
 0.68,
 1.91,
 0.68,
 0.68,
 1.22,
 0.68,
 0.68,
 0.68,
 1.22,
 0.68,
 0.68,
 0.68,
 0.68,
 0.68,
 0.68,
 0.68,
 1.21,
 0.68,
 0.68,
 1.02,
 0.68,
 2.22,
 0.68,
 0.68,
 0.68,
 0.68,
 0.68,
 0.68,
 1.11,
 0.68,
 1.95,
 1.21,
 1.21,
 0.68,
 1.01,
 0.68,
 0.68,
 0.68,
 1.22,
 0.68,
 0.68,
 0.68,
 0.68,
 1.11,
 1.91,
 0.68,
 1.22,
 0.68,
 0.68,
 1.11,
 0.68,
 0.68,
 0.68,
 1.91,
 1.95,
 1.21,
 0.68,
 0.68,
 0.68,
 0.68,
 1.01,
 1.11,
 1.95,
 0.68,
 0.68,
 0.68,
 1.11,
 0.68,
 1.11,
 1.95,
 1.11,
 1.22,
 1.11,
 0.68,
 0.68,
 2.22,
 1.22,
 0.68,
 1.21,
 0.68,
 1.11,
 1.92,
 0.68,
 0.68,
 1.95,
 1.91,
 0.68,
 0.68,
 0.68,
 0.68,
 1.95,
 0.68,
 1.01,
 0.68,
 0.68,
 0.68,
 1.11,
 1.01,
 0.68,
 0.68,
 0.68,
 0.68,
 1.91,
 0.68,
 1.01,
 1.95,
 0.68,
 0.68,
 0.68,
 0.68,
 1.21,
 0.68,
 0.68,
 0.68,
 0.68,
 1.11,

In [ ]:
ground_truths

[0.68,
 0.68,
 1.39,
 0.68,
 1.07,
 2.83,
 0.68,
 0.68,
 1.53,
 0.68,
 1.64,
 0.68,
 0.68,
 1.05,
 0.68,
 0.68,
 1.38,
 0.9,
 2.18,
 0.68,
 0.68,
 0.68,
 0.68,
 0.68,
 1.08,
 0.68,
 1.16,
 0.68,
 1.06,
 0.68,
 0.68,
 1.99,
 1.41,
 1.06,
 0.68,
 2.31,
 2.25,
 0.98,
 0.68,
 0.68,
 1.03,
 1.11,
 1.02,
 1.85,
 0.68,
 1.29,
 1.47,
 0.68,
 3.37,
 0.68,
 0.77,
 1.1,
 1.01,
 0.68,
 0.68,
 1.44,
 0.68,
 2.58,
 2.34,
 0.68,
 1.52,
 1.84,
 0.68,
 1.1,
 1.4,
 1.85,
 0.89,
 1.16,
 0.68,
 0.68,
 2.51,
 1.52,
 1.32,
 2.24,
 0.68,
 2.37,
 1.03,
 0.96,
 1.85,
 0.68,
 2.6,
 1.39,
 1.91,
 1.81,
 2.36,
 0.68,
 0.68,
 2.18,
 0.68,
 2.81,
 0.68,
 1.51,
 0.78,
 2.12,
 1.02,
 1.26,
 0.94,
 1.85,
 2.7,
 1.27,
 0.68,
 1.26,
 1.76,
 1.84,
 0.68,
 1.88,
 1.54,
 1.93,
 0.68,
 0.68,
 0.68,
 2.32,
 0.93,
 0.68,
 0.68,
 0.68,
 0.68,
 1.71,
 0.68,
 1.59,
 0.68,
 1.16,
 0.68,
 1.25,
 2.75,
 0.68,
 0.68,
 1.62,
 1.38,
 2.01,
 0.68,
 0.68,
 1.69,
 0.68,
 1.7,
 0.68,
 2.21,
 2.64,
 0.68,
 0.68,
 0.68,
 1.0,
 0.68,
 1.38,


In [ ]:
np.random.normal(0, 0.1)

NameError: name 'np' is not defined